In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime, date
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import statsmodels.api as sm
import numpy as np

In [2]:
def price_data(start_date, end_date, ticker, columns_list, timeframe):

    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    price = yf.download(ticker, start=start_date, end=end_date)
    price.index = pd.to_datetime(price.index)
    price = price[columns_list]
    price = pd.DataFrame(price)

    timeframe_settings = {'Close': 'last', 'Open': 'first'} #'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}
    if timeframe=="W":
        price = price.resample("W").agg(timeframe_settings)
    if timeframe=="M":
        price = price.resample("M").agg(timeframe_settings)
    if timeframe=="Y":
        price = price.resample("Y").agg(timeframe_settings)
    if timeframe=="D":
        pass
    return price

In [3]:
week_sentiment = pd.read_csv("./Datos ecopetrol/Datos_sentimiento_definitivos_EC.csv")
week_topics = pd.read_csv("./Datos ecopetrol/Datos_ecopetrol_topicos_modelos.csv")

try:
    week_sentiment = week_sentiment.drop(columns="Unnamed: 0")
    week_topics = week_topics.drop(columns="Unnamed: 0")
except:
    pass

# Join final data for models:
week_data = pd.merge(week_sentiment, week_topics)
week_data["Date"] = pd.to_datetime(week_data["Date"])

print(week_data["Date"].min())
print(week_data["Date"].max())

2012-07-01 00:00:00
2023-12-31 00:00:00


In [4]:
week_price = price_data(start_date="2012-07-01",end_date="2023-12-31",ticker="EC",columns_list=["Close","Open"],timeframe="W")

[*********************100%%**********************]  1 of 1 completed


In [5]:
week_price

,Close,Open
Date,,
2012-07-08,55.740002,56.669998
2012-07-15,55.910000,55.270000
2012-07-22,55.080002,55.799999
2012-07-29,57.520000,54.290001
2012-08-05,59.080002,57.240002
...,...,...
2023-12-03,12.810000,12.700000
2023-12-10,12.720000,12.700000
2023-12-17,12.800000,12.710000


In [5]:
# Calculating returns
week_price["diff"] = ((week_price['Close'] - week_price['Open']) / week_price['Open']) * 100

# Price trend
week_price["trend"] = (week_price['Close'] > week_price['Open']).astype(int)


# Getting open price of next week and close price after 1,2,3 and 4 weeks
week_price["open1"] = week_price["Open"].shift(-1)
week_price["close2"] = week_price["Close"].shift(-2)
week_price["close3"] = week_price["Close"].shift(-3)
week_price["close4"] = week_price["Close"].shift(-4)

# Categorical variables
week_price["trend1"] = week_price["trend"].shift(-1)
week_price["trend2"] = ((week_price['open1'] < week_price['close2'])).astype(int)
week_price["trend3"] = ((week_price['open1'] < week_price['close3'])).astype(int)
week_price["trend4"] = ((week_price['open1'] < week_price['close4'])).astype(int)

# Numerical variables "returns"
week_price["diff1"] = week_price["diff"].shift(-1)
week_price['diff2'] = ((week_price['close2'] - week_price['open1']) / week_price['open1']) * 100
week_price['diff3'] = ((week_price['close3'] - week_price['open1']) / week_price['open1']) * 100
week_price['diff4'] = ((week_price['close4'] - week_price['open1']) / week_price['open1']) * 100

In [6]:
week_price = week_price.dropna()

In [7]:
week_data = pd.merge(week_price, week_data, on='Date', how='left')
week_data["trend1"] = week_data["trend1"].astype(int)

In [8]:
week_data = week_data.reset_index()

In [11]:
week_data = week_data.drop(columns="index")

In [12]:
week_data.to_csv("./Datos ecopetrol/DATOS_PREDICTIVOS_EC.csv", index=False)